In [2]:
import os
import csv
import requests
import lxml
import time
import cloudscraper
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from transliterate import slugify
from tqdm import tqdm

In [3]:
def get_html(url_):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
    scraper = cloudscraper.create_scraper(delay=10, debug=False,  browser='chrome')
    response = scraper.get(url_, headers=headers)
    return response

def get_prods(url_):
    response = 0
    for i in range(10):
        try:
            response= get_html(url_)
            break
        except:
            time.sleep(1)
            continue
    return response

In [38]:
with open('download.xlsx', "rb") as f:
    df = pd.read_excel(f, dtype=str)
print(df.shape)
df.head()

(52, 4)


,_MODEL_,_SKU_,Name,Url
0,00000015089,ДАИ-18-2-Ли КНМ4,"Дрель-шуруповерт ""BRUSHLESS"" 18В 2АКБ ЗУБР Литий",https://www.vseinstrumenti.ru/instrument/shuru...
1,00000012714,"ДА-14,4-2-Ли КНМ1","Дрель-шуруповерт ""М1""14,4В 2АКБ ЗУБР Литий",https://www.vseinstrumenti.ru/instrument/shuru...
2,00000015086,"ДА-14,4-2-Ли КМ1","Дрель-шуруповерт ""М1""14,4В ЗУБР Литий",https://www.vseinstrumenti.ru/instrument/shuru...
3,00000013193,ДА-12-2-Ли КНМ1,Дрель-шуруповерт 12В 2АКБ ЗУБР Литий,https://www.vseinstrumenti.ru/instrument/shuru...
4,00000021381,CD-120-2,Дрель-шуруповерт STEHER 12В 2АКБ(2Ач),https://www.vseinstrumenti.ru/instrument/shuru...


In [39]:
df1 = pd.DataFrame(columns=['_MODEL_', '_SKU_', '_NAME_', '_CATEGORY_', '_MAIN_CATEGORY_', '_MANUFACTURER_', '_DESCRIPTION_', '_ATTRIBUTES_', '_PRODUCT_IMAGES_', '_STATUS_', '_SEO_KEYWORD_'])
df1.to_excel("opencart.xlsx", index = False)

df1 = pd.DataFrame(columns=['_MODEL_', '_SKU_', 'Name', 'Url', 'Where', 'status_code', 'reason'])
df1.to_excel("error.xlsx", index = False)

In [40]:
for index in tqdm(df.index):
    response = get_prods(df['Url'][index])
    if(response.status_code==200):
        soup = BeautifulSoup(response.text, "lxml")

        _NAME_ = soup.find('h1').text.strip()
        _SEO_KEYWORD_ = slugify(_NAME_)

        breadcrumbs = soup.find('nav', class_="breadcrumbs").text.split('/')
        breadcrumbs = [s.strip() for s in breadcrumbs]
        _CATEGORY_ = '|'.join(map(str, breadcrumbs[1:]))
        _MAIN_CATEGORY_ = _CATEGORY_
        _MANUFACTURER_ = soup.find('img', class_="brand-logo")['alt']
        _DESCRIPTION_ = soup.find(itemprop="description").text.strip()

        # Find attributes
        tag_names = soup.find(class_='features spoiler').findAll(itemprop='name')
        tag_values = soup.find(class_='features spoiler').findAll(itemprop='value')
        _ATTRIBUTES_ = ''
        for i in range(len(tag_names)):
            _ATTRIBUTES_ = _ATTRIBUTES_ + 'Характеристики' + '|' + tag_names[i].text.strip() + '|' + tag_values[i].text.strip() + '\n'
        
        # Find images link
        images = soup.find(class_='listing-carousel').find_all('a')
        big_img = []
        for img in images:
            zoom = img.find('div', class_='zoom')
            responses = zoom['style'].split("url('")[1][:-2]
            if(zoom.find_next_sibling('div')==None):
                big_img.append(responses)
        
        # Download images
        local_path = []
        for url_img in big_img:
            resp = get_prods(url_img)
            path = 'catalog/' + '/'.join(map(str, url_img.split('/')[4:]))
            local_path.append(path)
            # Save the image
            if (type(resp)!=int and resp.status_code == 200):
                os.makedirs(os.path.dirname(path), exist_ok=True)
                with open(path, "wb") as f:
                    f.write(resp.content)
            else:
                df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Images', resp.status_code, resp.reason]])
                with pd.ExcelWriter('error.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
                    df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)
            
            _PRODUCT_IMAGES_ = ','.join(map(str, local_path))
        
        _STATUS_ = '1'
        
        df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], _NAME_, _CATEGORY_, _MAIN_CATEGORY_, _MANUFACTURER_, _DESCRIPTION_, _ATTRIBUTES_, _PRODUCT_IMAGES_, _STATUS_, _SEO_KEYWORD_]])
        with pd.ExcelWriter('opencart.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
            df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)
    else:
        df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Url', response.status_code, response.reason]])
        with pd.ExcelWriter('error.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
            df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)

100%|██████████| 52/52 [03:53<00:00,  4.49s/it]


In [ ]:
with open('opencart.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_ALL)   
  write.writerow(['_MODEL_', '_SKU_', '_NAME_', '_CATEGORY_', '_MAIN_CATEGORY_', '_MANUFACTURER_', '_DESCRIPTION_', '_ATTRIBUTES_', '_PRODUCT_IMAGES_', '_STATUS_', '_SEO_KEYWORD_'])

with open('error.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_ALL)   
  write.writerow(['_MODEL_', '_SKU_', 'Name', 'Url', 'Where', 'status_code', 'reason'])

In [ ]:
with open('download.xlsx', "rb") as f:
    df = pd.read_excel(f, dtype=str)
df

,_MODEL_,_SKU_,Name,Url
0,00000015295,УШМ-115-800 М3,"Углошлифовальная машина ЗУБР УШМ-115-800 М3, 1...",https://www.vseinstrumenti.ru/instrument/shlif...
1,00000015296,УШМ-125-1100 ТМ3,Углошлифовальная машина ЗУБР УШМ-125-1100 ТМ3 ...,https://www.vseinstrumenti.ru/instrument/shlif...
2,00000015302,УШМ-125-1200 ЭМЗ,"Углошлифовальная машина ЗУБР УШМ-125-1200, 125...",https://www.vseinstrumenti.ru/instrument/shlif...


In [ ]:
for index in tqdm(df.index):
    response = get_prods(df['Url'][index])
    if(response.status_code==200):
        soup = BeautifulSoup(response.text, "lxml")

        _NAME_ = soup.find('h1').text.strip()
        _SEO_KEYWORD_ = slugify(_NAME_)

        breadcrumbs = soup.find('nav', class_="breadcrumbs").text.split('/')
        breadcrumbs = [s.strip() for s in breadcrumbs]
        _CATEGORY_ = '|'.join(map(str, breadcrumbs[1:]))
        _MAIN_CATEGORY_ = _CATEGORY_
        _MANUFACTURER_ = soup.find('img', class_="brand-logo")['alt']
        _DESCRIPTION_ = soup.find(itemprop="description").text.strip()

        # Find attributes
        tag_names = soup.find(class_='features spoiler').findAll(itemprop='name')
        tag_values = soup.find(class_='features spoiler').findAll(itemprop='value')
        _ATTRIBUTES_ = ''
        for i in range(len(tag_names)):
            _ATTRIBUTES_ = _ATTRIBUTES_ + 'Характеристики' + '|' + tag_names[i].text.strip() + '|' + tag_values[i].text.strip() + '\n'
        
        # Find images link
        images = soup.find(class_='listing-carousel').find_all('a')
        big_img = []
        for img in images:
            zoom = img.find('div', class_='zoom')
            responses = zoom['style'].split("url('")[1][:-2]
            if(zoom.find_next_sibling('div')==None):
                big_img.append(responses)
        
        # Download images
        local_path = []
        for url_img in big_img:
            resp = get_prods(url_img)
            path = 'catalog/' + '/'.join(map(str, url_img.split('/')[4:]))
            local_path.append(path)
            # Save the image
            if resp.status_code == 200:
                os.makedirs(os.path.dirname(path), exist_ok=True)
                with open(path, "wb") as f:
                    f.write(resp.content)
            else:
                with open('error.csv', 'a', newline='') as f:
                    write = csv.writer(f, delimiter=';')   
                    write.writerow([df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Images', resp.status_code, resp.reason])
            
            _PRODUCT_IMAGES_ = ','.join(map(str, local_path))
        
        _STATUS_ = '1'

        with open('opencart.csv', 'a', newline='', encoding="utf-8") as f:
            write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONNUMERIC)   
            write.writerow([df['_MODEL_'][index], df['_SKU_'][index], _NAME_, _CATEGORY_, _MAIN_CATEGORY_, _MANUFACTURER_, _DESCRIPTION_, _ATTRIBUTES_, _PRODUCT_IMAGES_, _STATUS_, _SEO_KEYWORD_])
    else:
        with open('error.csv', 'a', newline='', encoding="utf-8") as f:
            write = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONNUMERIC)   
            write.writerow([df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Url', response.status_code, response.reason])

100%|██████████| 3/3 [00:11<00:00,  3.90s/it]
